In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Reshape, Conv1D, Conv2D, Add, BatchNormalization, MaxPooling1D)

In [19]:
def res_layer(filter, input):
    aconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(input)
    bconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(aconv1d)
    cconv1d = Conv1D(filters=filter, kernel_size=1, padding='same')(bconv1d)
    normalized = BatchNormalization()(cconv1d)
    raconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(normalized)
    lconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(normalized)
    rbconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(raconv1d)
    rcconv1d = Conv1D(filters=filter, kernel_size=1, padding='same')(rbconv1d)
    rmaxpooling = MaxPooling1D(pool_size=2)(rcconv1d)
    lmaxpooling = MaxPooling1D(pool_size=2)(lconv1d)
    add = Add()([rmaxpooling, lmaxpooling])
    return add

def res_layer_wopooling(filter, input):
    aconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(input)
    bconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(aconv1d)
    cconv1d = Conv1D(filters=filter, kernel_size=1, padding='same')(bconv1d)
    normalized = BatchNormalization()(cconv1d)
    return normalized


def pooling_layer(filter, input):
    raconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(input)
    lconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(input)
    rbconv1d = Conv1D(filters=filter, kernel_size=3, padding='same')(raconv1d)
    rcconv1d = Conv1D(filters=filter, kernel_size=1, padding='same')(rbconv1d)
    rmaxpooling = MaxPooling1D(pool_size=2)(rcconv1d)
    lmaxpooling = MaxPooling1D(pool_size=2)(lconv1d)
    add = Add()([rmaxpooling, lmaxpooling])
    return add

In [20]:

#embedding
myInput = Input(shape=(20,1912))
reshaped = Reshape(target_shape=(20,1912,1))(myInput)
myConv2d = Conv2D(filters=256, kernel_size=(20,1))(reshaped)
reshaped = Reshape(target_shape=(1912,256))(myConv2d)
normalized = BatchNormalization()(reshaped)


model = Model(myInput, normalized)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20, 1912)          0         
_________________________________________________________________
reshape_10 (Reshape)         (None, 20, 1912, 1)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 1912, 256)      5376      
_________________________________________________________________
reshape_11 (Reshape)         (None, 1912, 256)         0         
_________________________________________________________________
batch_normalization_v1_54 (B (None, 1912, 256)         1024      
Total params: 6,400
Trainable params: 5,888
Non-trainable params: 512
_________________________________________________________________


In [21]:
temp=res_layer(256, normalized)
temp=res_layer(128, temp)
temp=res_layer(64, temp)
temp=res_layer(32, temp)
temp=res_layer(16, temp)
temp=res_layer(8, temp)
temp=res_layer(4, temp)
temp=res_layer_wopooling(2, temp)
temp=res_layer_wopooling(1, temp)
temp=pooling_layer(1, temp)
model = Model(myInput, temp)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 20, 1912)     0                                            
__________________________________________________________________________________________________
reshape_10 (Reshape)            (None, 20, 1912, 1)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 1, 1912, 256) 5376        reshape_10[0][0]                 
__________________________________________________________________________________________________
reshape_11 (Reshape)            (None, 1912, 256)    0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
batch_norm